In [1]:
#!pip install transformers
#!pip install keybert
#pip install auto_gptq==0.2.0
import torch
import pandas as pd
import pandas as pd
from transformers import pipeline
from concurrent.futures import ThreadPoolExecutor
import time

print(torch.cuda.is_available())

d:\Anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


True


In [36]:
# zero-shot learning pipeline, summarization models available on huggingface - https://huggingface.co/models?pipeline_tag=summarization&sort=downloads

from transformers import pipeline
device = 0 if torch.cuda.is_available() else -1

summarizer_facebook_bart = pipeline("summarization", model="facebook/bart-large-cnn",device=device) # https://huggingface.co/facebook/bart-large-cnn
# classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli",device=device))
summarizer_philschmid_bart = pipeline("summarization", model="philschmid/bart-large-cnn-samsum",device=device) # https://huggingface.co/philschmid/bart-large-cnn-samsum
summarizer_google_pegasus = pipeline("summarization", model="google/pegasus-xsum",device=device) # https://huggingface.co/google/pegasus-xsum
summarizer_falconsai = pipeline("summarization", model="Falconsai/text_summarization",device=device) # https://huggingface.co/Falconsai/text_summarization
summarizer_distilbart = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6",device=device)
# summarizer_google_pegasusL = pipeline("summarization", model="google/pegasus-large",device=device)) # https://huggingface.co/google/pegasus-large
# summarizer_sshleifer_distilbart_6_6 = pipeline("summarization", model="sshleifer/distilbart-cnn-6-6",device=device)) # https://huggingface.co/sshleifer/distilbart-cnn-6-6
summarizer_azma_bart = pipeline("summarization", model="Azma-AI/bart-large-text-summarizer",device=device) # https://huggingface.co/Azma-AI/bart-large-text-summarizer

list_of_summarizer = [summarizer_facebook_bart,summarizer_falconsai,summarizer_google_pegasus,summarizer_azma_bart,summarizer_distilbart,summarizer_philschmid_bart]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
if torch.cuda.is_available(): device = 0  
else: device = -1  
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  12341 MiB |  15810 MiB | 134870 MiB | 122528 MiB |
|       from large pool |  11751 MiB |  14619 MiB |  92983 MiB |  81231 MiB |
|       from small pool |    590 MiB |   1190 MiB |  41887 MiB |  41297 MiB |
|---------------------------------------------------------------------------|
| Active memory         |  12341 MiB |  15810 MiB | 134870 MiB | 122528 MiB |
|       from large pool |  11751 MiB |  14619 MiB |  92983 MiB |

In [ ]:
#testing with llama13B
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from transformers import GPTQConfig

device = 0 if torch.cuda.is_available() else -1
quantization_config = GPTQConfig(bits=4, disable_exllama=True)

model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/WizardLM-1.0-Uncensored-Llama2-13B-GPTQ",
    quantization_config=quantization_config
).to('cuda')

tokenizer = AutoTokenizer.from_pretrained("TheBloke/WizardLM-1.0-Uncensored-Llama2-13B-GPTQ")

# Create the pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)


LLM summarization model (pretrained)

In [5]:
description = """principal activity is that of a the following items are recognised in OCI: property developer and the place of business is in Singapore. The Group is also a 20% (2021: 20%) partner in Park Court Aoyama The Tower,
a joint arrangement formed with a third party, whose principal activity is that of a   differences arising on the translation of monetary items that in substance form part of the Group s net property developer and the place of business is in Japan.
The Group has classified both joint arrangements as joint investment in a foreign operation; operations as the joint venture partners control the joint arrangements collectively and the joint arrangements are not structured through separate legal vehicles.
   an investment in equity securities designated at fair value through other comprehensive income (FVOCI); (vi) Transactions eliminated on consolidation
   a financial liability designated as a hedge of the net investment in a foreign operation to the extent that the hedge is effective; or Intra-group balances and transactions, and any unrealised income or expenses arising from intra-group transactions,
   are eliminated in preparing the consolidated financial statements. Unrealised gains arising from   qualifying cash flow hedges to the extent that the hedges are effective. transactions with equity-accounted investee are eliminated against the investment
    to the extent of the Group s interest in the investee. Unrealised losses are eliminated in the same way as unrealised gains, but only to the (ii) Foreign operations extent that there is no evidence of impairment. The assets and liabilities of foreign
    operations, including goodwill and fair value adjustments arising on acquisition, (vii) Subsidiaries, associates and joint ventures in the separate financial statements are translated to Singapore dollars at exchange rates prevailing at the reporting
    date. The income and expenses of foreign operations are translated to Singapore dollars at exchange rates at the dates of the transactions. Investments in subsidiaries, associates and joint ventures are stated in the Company s statement of financial
    position at cost less accumulated impairment losses.
Foreign currency differences are recognised in OCI. However, if the foreign operation is a non wholly-owned subsidiary, then the relevant proportionate share of the translation difference is allocated to the non-controlling 3."""
              
#Summarize the description
summary = summarizer_facebook_bart(description, max_length=150, min_length=30, do_sample=False)
print(summary[0]['summary_text'])

from keybert import KeyBERT
kw_model = KeyBERT()

# Extract keywords from the summarized description
keywords = kw_model.extract_keywords(summary[0]['summary_text'])
print(keywords)

d:\Anaconda\lib\site-packages\transformers\models\bart\modeling_bart.py:603: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


The Group has classified both joint arrangements as joint investment in a foreign operation. The Group is also a 20% (2021: 20%) partner in Park Court Aoyama The Tower, a joint arrangement formed with a third party.
[('joint', 0.4634), ('aoyama', 0.3924), ('arrangement', 0.357), ('partner', 0.3238), ('arrangements', 0.3205)]


In [78]:
#BRC ASIA LIMITED 2022
#Principal Activity SSIC Code 24109
#MANUFACTURE OF BASIC IRON AND STEEL N.E.C. (INCLUDING SMELTING)

# description = """principal activities of the Company are the prefabrication of steel reinforcement for use in concrete, 
# trading of steel reinforcing bars, and manufacturing and sale of wire mesh fences. 
# The principal activities of the subsidiaries are disclosed in Note 13 to the fi nancial statements. 
# """

description = """ the Company are those of print and paper management related activities and investment holding. 
the subsidiaries are those of trading of commodity chemical products, 
provision of water treatment solutions using microbial and/or chemicals in the People s Republic of China ( PRC ), 
print and paper management related activities and in investment and trading of securities.
"""

#CITY DEVELOPMENTS LIMITED 
#Principal Activity SSIC Code 68101 REAL ESTATE DEVELOPERS
# description = """principal activity is that of a the following items are recognised in OCI: property developer and the place of business is in Singapore. The Group is also a 20% (2021: 20%) partner in Park Court Aoyama The Tower,
# a joint arrangement formed with a third party, whose principal activity is that of a   differences arising on the translation of monetary items that in substance form part of the Group s net property developer and the place of business is in Japan.
# The Group has classified both joint arrangements as joint investment in a foreign operation; operations as the joint venture partners control the joint arrangements collectively and the joint arrangements are not structured through separate legal vehicles.
#    an investment in equity securities designated at fair value through other comprehensive income (FVOCI); (vi) Transactions eliminated on consolidation
#    a financial liability designated as a hedge of the net investment in a foreign operation to the extent that the hedge is effective; or Intra-group balances and transactions, and any unrealised income or expenses arising from intra-group transactions,
#    are eliminated in preparing the consolidated financial statements. Unrealised gains arising from   qualifying cash flow hedges to the extent that the hedges are effective. transactions with equity-accounted investee are eliminated against the investment
#     to the extent of the Group s interest in the investee. Unrealised losses are eliminated in the same way as unrealised gains, but only to the (ii) Foreign operations extent that there is no evidence of impairment. The assets and liabilities of foreign
#     operations, including goodwill and fair value adjustments arising on acquisition, (vii) Subsidiaries, associates and joint ventures in the separate financial statements are translated to Singapore dollars at exchange rates prevailing at the reporting
#     date. The income and expenses of foreign operations are translated to Singapore dollars at exchange rates at the dates of the transactions. Investments in subsidiaries, associates and joint ventures are stated in the Company s statement of financial
#     position at cost less accumulated impairment losses.
# Foreign currency differences are recognised in OCI. However, if the foreign operation is a non wholly-owned subsidiary, then the relevant proportionate share of the translation difference is allocated to the non-controlling 3."""
       
    
for summarizer in list_of_summarizer:
        model_name = summarizer.model.name_or_path if hasattr(summarizer, 'model') else summarizer.__name__
        summary = summarizer(description, max_length=max(30, int(len(description)/6)), min_length=30, do_sample=False)
        print(f"{model_name}: {summary[0]['summary_text']}")


#philschmid/bart-large-cnn-samsum - perform the best

facebook/bart-large-cnn: The Company is engaged in print and paper management related activities and investment holding. The subsidiaries are those of trading of commodity chemical products,  provision of water treatment solutions using microbial and/or chemicals in the People s Republic of China.
Falconsai/text_summarization: the subsidiaries are those of trading of commodity chemical products, provision of water treatment solutions using microbial and/or chemicals in the People s Republic of China .
google/pegasus-xsum: The board of directors of Anhui Conch Paper Group Company Limited (the "Company"), a company incorporated in the People s Republic of China ( the "PRC"), has declared a quarterly dividend of 5 cents per share to the shareholders of record as of the close of business on the 25th
Azma-AI/bart-large-text-summarizer: The Company is engaged in print and paper management related activities and investment holding. The subsidiaries are those of trading of commodity chemical p

In [31]:
#Files required for summarization
df_linkedIn=pd.read_excel("../dataSources/ScrapedOutputFiles/linkedInScrapedOutputs.xlsx")
df_test=df_linkedIn.head(5)

#SSIC Ref detailes
SSIC_ref_df = pd.read_csv('SSIC Fact Ref Table.csv')
SSIC_ref_df_test=SSIC_ref_df.head(10)
#SSIC_ref_df_test
#df_test

,PDF Name,Page Number,Notes Page Content
0,ABR HOLDINGS LIMITED 2022.pdf,74,principal activities of the Company are the ma...
1,ABUNDANCE INTERNATIONAL LIMITED 2022.pdf,56,principal activities of the Company are those ...
2,ABUNDANTE LIMITED 2023.pdf,48,principal activities of the Company are those ...
3,ACCRELIST LTD. 2023.pdf,55,principal activities of the Company is investm...
4,ACESIAN PARTNERS LIMITED 2022.pdf,54,principal activities of the Group consist of d...


In [34]:
# Summarize the descriptions
df_linkedIn['Summarized_Description'] = df_linkedIn['Notes Page Content'].apply(
    lambda x: summarizer_azma_bart(x, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
)

# 85 rows - 5m 41s

Your max_length is set to 150, but your input_length is only 56. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)
Your max_length is set to 150, but your input_length is only 94. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)
Your max_length is set to 150, but your input_length is only 91. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)
Your max_length is set to 150, but your input_length is only 22. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)
Your

In [35]:
df_linkedIn.to_excel("Summarised_input.xlsx")

In [39]:
df_linkedIn

,PDF Name,Page Number,Notes Page Content,Summarized_Description
0,ABR HOLDINGS LIMITED 2022.pdf,74,principal activities of the Company are the ma...,The principal activities of the Company are th...
1,ABUNDANCE INTERNATIONAL LIMITED 2022.pdf,56,principal activities of the Company are those ...,The principal activities of the Company are th...
2,ABUNDANTE LIMITED 2023.pdf,48,principal activities of the Company are those ...,The Company's ultimate controlling party is Le...
3,ACCRELIST LTD. 2023.pdf,55,principal activities of the Company is investm...,The principal activities of the Company are in...
4,ACESIAN PARTNERS LIMITED 2022.pdf,54,principal activities of the Group consist of d...,The principal activities of the Group consist ...
...,...,...,...,...
80,TELECHOICE INTERNATIONAL LIMITED 2022.pdf,129,principal activities of the Company during the...,The principal activities of the Company during...
81,TT INTERNATIONAL LIMITED 2023.pdf,56,principal activities of the Company are those ...,The principal activities of the Company are th...
82,UNITED OVERSEAS INSURANCE LIMITED 2022.pdf,4,principal activities are the underwriting 1,The company's principal activities are the und...
83,VENTURE CORPORATION LIMITED 2022.pdf,107,principal activities of the Company are to pro...,"The Company provides manufacturing, product de..."


In [42]:
SSIC_ref_df = SSIC_ref_df.drop_duplicates(subset=['SSIC 2020'], keep='first')
SSIC_ref_df

,SSIC 2020,SSIC 2020 Title,Detailed Definitions,Cross References,Examples of Activities Classified Under this Code,"Section, 2 digit code",Division,Group,Class,Section,Section Title,Division Title,Group Title,Class Title
0,1111,Growing of leafy and fruit vegetables,This sub-class includes the cultivation of lea...,"•growing of mushrooms, see 01112\r\n•growing o...",NaN,1,1,11,111,A,AGRICULTURE AND FISHING,AGRICULTURE AND RELATED SERVICE ACTIVITIES,"GROWING OF CROPS, MARKET GARDENING AND HORTICU...",Growing of Food Crops (Non-Hydroponics)
1,1112,Growing of mushrooms,This sub-class includes the cultivation of mus...,•growing of leafy and fruit vegetables (non-hy...,NaN,1,1,11,111,A,AGRICULTURE AND FISHING,AGRICULTURE AND RELATED SERVICE ACTIVITIES,"GROWING OF CROPS, MARKET GARDENING AND HORTICU...",Growing of Food Crops (Non-Hydroponics)
2,1113,Growing of root crops,This sub-class includes the cultivation of roo...,•growing of leafy and fruit vegetables (non-hy...,NaN,1,1,11,111,A,AGRICULTURE AND FISHING,AGRICULTURE AND RELATED SERVICE ACTIVITIES,"GROWING OF CROPS, MARKET GARDENING AND HORTICU...",Growing of Food Crops (Non-Hydroponics)
3,1119,Growing of food crops (non-hydroponics) n.e.c.,This sub-class includes the propagation of foo...,•growing of leafy and fruit vegetables (non-hy...,•cereals growing (non-hydroponics)\r\n•rice gr...,1,1,11,111,A,AGRICULTURE AND FISHING,AGRICULTURE AND RELATED SERVICE ACTIVITIES,"GROWING OF CROPS, MARKET GARDENING AND HORTICU...",Growing of Food Crops (Non-Hydroponics)
4,1120,Growing of leafy and fruit vegetables (hydropo...,This sub-class includes the cultivation of lea...,•growing of leafy and fruit vegetables (non-hy...,NaN,1,1,11,112,A,AGRICULTURE AND FISHING,AGRICULTURE AND RELATED SERVICE ACTIVITIES,"GROWING OF CROPS, MARKET GARDENING AND HORTICU...",Growing of Food Crops (Hydroponics)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,97001,Activities of households as employers of domes...,This Sub-class includes activities of househol...,•activities of households as employers of othe...,NaN,97,97,970,9700,T,ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS OF DOMES...,ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS OF DOMES...,ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS OF DOMES...,Activities of Households as Employers of Domes...
1019,97002,Activities of households as employers of other...,This Sub-class includes activities of househol...,•activities of households as employers of dome...,NaN,97,97,970,9700,T,ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS OF DOMES...,ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS OF DOMES...,ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS OF DOMES...,Activities of Households as Employers of Domes...
1020,99010,Foreign embassies and trade representative off...,This Sub-class includes activities of: \r\n•di...,NaN,NaN,99,99,990,9901,U,ACTIVITIES OF EXTRA-TERRITORIAL ORGANISATIONS ...,ACTIVITIES OF EXTRA-TERRITORIAL ORGANISATIONS ...,ACTIVITIES OF EXTRA-TERRITORIAL ORGANISATIONS ...,Foreign Embassies and Trade Representative Off...
1021,99020,Foreign armed forces,This Sub-class includes activities of foreign ...,NaN,NaN,99,99,990,9902,U,ACTIVITIES OF EXTRA-TERRITORIAL ORGANISATIONS ...,ACTIVITIES OF EXTRA-TERRITORIAL ORGANISATIONS ...,ACTIVITIES OF EXTRA-TERRITORIAL ORGANISATIONS ...,Foreign Armed Forces


KeyBert
https://github.com/MaartenGr/KeyBERT

Embedding documents; 
Creating candidate keywords; 
Calculating best keywords through either MMR, Max Sum Similarity, or Cosine Similarity

In [77]:
from keybert import KeyBERT

# description = """principal activities of the Company are the prefabrication of steel reinforcement for use in concrete, 
# trading of steel reinforcing bars, and manufacturing and sale of wire mesh fences. 
# The principal activities of the subsidiaries are disclosed in Note 13 to the financial statements. 
# """
description = """principal activities of the Company are those of print and paper management related activities and investment holding. 
The principal activities of the subsidiaries are those of trading of commodity chemical products, 
provision of water treatment solutions using microbial and/or chemicals in the People s Republic of China ( PRC ), 
print and paper management related activities and in investment and trading of securities.
"""

#keyphrase_ngram_range ---->  depending on the number of words in the resulting keyphrases
keywords = KeyBERT().extract_keywords(description,keyphrase_ngram_range=(2, 2),top_n=10)
keywords

[('principal activities', 0.6735),
 ('activities subsidiaries', 0.5877),
 ('activities company', 0.4743),
 ('subsidiaries trading', 0.4476),
 ('activities investment', 0.409),
 ('holding principal', 0.3914),
 ('chemical products', 0.3815),
 ('company print', 0.3769),
 ('paper management', 0.3719),
 ('related activities', 0.3711)]

In [ ]:
# apply keybert on ref table, change the range accordinly
SSIC_ref_df['Keywords'] = SSIC_ref_df['Detailed Definitions'].apply(
    lambda x: kw_model.extract_keywords(x, keyphrase_ngram_range=(2, 2))
)

SSIC_ref_df['Keywords']
#return: keywords: The top n keywords for a document with their respective distancesto the input document.
SSIC_ref_df.to_excel("SSIC Fact Ref_with keywords.xlsx")

In [63]:
#compare keywords got from description
for _, row in SSIC_ref_df.iterrows():
    fact_keywords = [kw[0] for kw in row['Keywords']]
    new_submission_keywords = [kw[0] for kw in keywords]
    match = set(fact_keywords).intersection(set(new_submission_keywords))
    
    if match:
        print(f"Matching SSIC: {row['SSIC 2020']}, Keywords: {match}")

Embedding comparison

In [67]:
from sentence_transformers import SentenceTransformer, util

#pretained minil,
model = SentenceTransformer('all-MiniLM-L6-v2')

# Convert the summarized descriptions and SSIC definitions to embeddings
test_linkedin=df_linkedIn.head(10)
test_ssicref=SSIC_ref_df.head(10)
test_linkedin['Embeddings'] = (test_linkedin.head(10))['Summarized_Description'].apply(lambda x: model.encode(x))
test_ssicref['Embeddings'] = test_ssicref['Detailed Definitions'].apply(lambda x: model.encode(x))


def find_best_ssic_match(company_embedding, fact_table):
    best_match = None
    best_similarity = -1
    best_definition = None

    for _, row in fact_table.iterrows():
        similarity = util.pytorch_cos_sim(company_embedding, row['Embeddings'])[0][0].item()
        
        if similarity > best_similarity:
            best_similarity = similarity
            best_match = row['SSIC 2020']
            best_definition = row['Detailed Definitions']
    
    return best_match, best_similarity, best_definition

# Find the best matching SSIC code and its detailed definition for each company
matches = test_linkedin['Embeddings'].apply(lambda x: find_best_ssic_match(x, test_ssicref))

# Extract the best match SSIC and detailed definition
test_linkedin['Best_Match_SSIC'] = matches.apply(lambda x: x[0])
test_linkedin['Best_Match_Definition'] = matches.apply(lambda x: x[2])

test_linkedin

C:\Users\Admin\AppData\Local\Temp\ipykernel_6884\3277016640.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_linkedin['Embeddings'] = (test_linkedin.head(10))['Summarized_Description'].apply(lambda x: model.encode(x))
C:\Users\Admin\AppData\Local\Temp\ipykernel_6884\3277016640.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_ssicref['Embeddings'] = test_ssicref['Detailed Definitions'].apply(lambda x: model.encode(x))
C:\Users\Admin\AppData\Local\Temp\ipykernel_6884\3277016640.py:32: Setting

,PDF Name,Page Number,Notes Page Content,Summarized_Description,Embeddings,Best_Match_SSIC,Best_Match_Definition
0,ABR HOLDINGS LIMITED 2022.pdf,74,principal activities of the Company are the ma...,The principal activities of the Company are th...,"[-0.020132285, -0.10326839, -0.01515015, 0.012...",1120,This sub-class includes the cultivation of lea...
1,ABUNDANCE INTERNATIONAL LIMITED 2022.pdf,56,principal activities of the Company are those ...,The principal activities of the Company are th...,"[-0.033344787, -0.049288258, 0.022993648, -0.0...",1190,This Sub-class includes the growing of other c...
2,ABUNDANTE LIMITED 2023.pdf,48,principal activities of the Company are those ...,The Company's ultimate controlling party is Le...,"[-0.07526265, -0.040298507, 0.029111238, 0.032...",1111,This sub-class includes the cultivation of lea...
3,ACCRELIST LTD. 2023.pdf,55,principal activities of the Company is investm...,The principal activities of the Company are in...,"[-0.0043711183, -0.050210413, -0.010950963, -0...",1190,This Sub-class includes the growing of other c...
4,ACESIAN PARTNERS LIMITED 2022.pdf,54,principal activities of the Group consist of d...,The principal activities of the Group consist ...,"[-0.0027161513, 0.0018759626, -0.027635382, -0...",1190,This Sub-class includes the growing of other c...
5,ACMA LTD. 2022.pdf,48,principal activity of the Company is that of i...,The principal activity of Acma Ltd. is that of...,"[-0.046834115, 0.02470865, -0.029877186, -0.02...",1190,This Sub-class includes the growing of other c...
6,ADVANCED SYSTEMS AUTOMATION LIMITED 2022.pdf,52,principal activity of the Company is investmen...,The principal activity of the Company is inves...,"[-0.07500034, -0.021864938, -0.04694008, 0.070...",1190,This Sub-class includes the growing of other c...
7,ALLIANCE HEALTHCARE GROUP LIMITED 2023.pdf,82,principal activities of the company is that of...,The principal activities of the company are in...,"[-0.06722844, -0.008623031, -0.03112161, -0.01...",1190,This Sub-class includes the growing of other c...
8,ANNAIK LIMITED 2022.pdf,71,principal activity of the Company is that of i...,The principal activity of the Company is that ...,"[-0.029021166, -0.020046683, 0.020063434, -0.0...",1190,This Sub-class includes the growing of other c...
9,AP OIL INTERNATIONAL LIMITED 2022.pdf,61,The The Company is an investment holding comp...,The Company is an investment holding company. ...,"[-0.05421597, -0.0050037336, 0.006176454, 0.00...",1130,This Sub-class includes growing of fruits such...


In [74]:
test_row_index = 1  
test_description = test_linkedin.iloc[test_row_index]['Notes Page Content']
test_summary = summarizer(test_description, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
print("Summary:", test_summary)

Your max_length is set to 150, but your input_length is only 94. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)


Summary: The principal activities of the Company are those of print and paper management related activities and investment holding. The subsidiaries include trading of commodity chemical products, provision of water treatment solutions using microbial and/or chemicals in the People's Republic of China and in investment and trading of securities.


In [75]:
test_embedding = model.encode(test_summary)
SSIC_ref_df['Embeddings'] = SSIC_ref_df['Detailed Definitions'].apply(lambda x: model.encode(x))

def find_top_ssic_matches(test_embedding, fact_table_embeddings):
    similarities = []

    for _, row in fact_table_embeddings.iterrows():
        similarity = util.pytorch_cos_sim(test_embedding, row['Embeddings'])[0][0].item()
        similarities.append((row['SSIC 2020'], similarity, row['Detailed Definitions']))

    topN = sorted(similarities, key=lambda x: x[1], reverse=True)[:3] #n=3
    return topN

top_matches = find_top_ssic_matches(test_embedding, SSIC_ref_df)

for match in top_matches:
    print(f"SSIC Code: {match[0]}, Similarity: {match[1]}, Definition: {match[2]}")

C:\Users\Admin\AppData\Local\Temp\ipykernel_6884\3155042889.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SSIC_ref_df['Embeddings'] = SSIC_ref_df['Detailed Definitions'].apply(lambda x: model.encode(x))


SSIC Code: 94110, Similarity: 0.4943753480911255, Definition: This Sub-class includes:
•activities of organisations whose members’ interests centre on the development and prosperity of enterprises in a particular line of business or trade, including farming, or on the economic growth and climate of a particular geographical area or political sub-division without regard for the line of business
•activities of federations of such associations
•activities of chambers of commerce, guilds and similar organisations
•dissemination of information, representation before government agencies, public relations and labour negotiations of business and employer organisations
SSIC Code: 64201, Similarity: 0.49186697602272034, Definition: This Sub-class includes the activities of bank/financial holding companies, i.e. units that hold the assets (owning controlling-levels of equity) of a group of subsidiary corporations and whose principal activity is owning the group. The holding companies in this Sub-